In [1]:
import re

import parse
import pdfplumber
import pandas as pd
from collections import namedtuple

pd.set_option('display.max_rows', None)  # or 1000

In [2]:
directory = '2022_2_BIM_CDT'

files = ['{directory}/3A Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3B Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3C Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3D Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3E Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3F Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3G Oficial 2Bim.pdf'.format(directory = directory),
        '{directory}/3H Oficial 2Bim.pdf'.format(directory = directory),
        '{directory}/3I Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3J Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3K Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3L Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3M Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3N Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3O Oficial 2 Bim.pdf'.format(directory = directory),
        '{directory}/3P Oficial 2 Bim.pdf'.format(directory = directory)]
        


In [3]:
Line = namedtuple('Line', 'Nome')

In [4]:
name_re = re.compile(r'(Nome:) (.*) (Data)')
name_re2 = re.compile(r'Nome:(.*)Data')
ra_re = re.compile(r'(RA:) (.*) (RM:)')


In [5]:
base_df = pd.DataFrame(columns=['TURMA', 'NOME', 'RA', 'MEDIA'])

for file in files:
    stu_class = file[15:17]
    
    with pdfplumber.open(file) as pdf:

        for page in pdf.pages[0:]:
            table = page.extract_table(table_settings={"vertical_strategy": "lines", 
                                                       "horizontal_strategy": "lines", 
                                                       "snap_tolerance": 3})

            ################# EXTRAÇÃO DE NOMES E RA #####################

            text = page.extract_text()

            for line in text.split('\n'):
                name = name_re.search(line)
                name2 = name_re2.search(line.replace(" ", ""))
                
                ra = ra_re.search(line)

                if name:
                    name_A = name.group(2)
                    
                elif name2:
                    name_A = name2.group(1)

                if ra:
                    ra_A = str(ra.group(2))


            ################# LÓGICA DE NOTAS ###################

            df = pd.DataFrame(table, columns=table[0]).T
            
            #### OS BIMESTRES SÃO SELECIONADOS AQUI ###
            df1 = df.iloc[3:, 10:12].replace("ES", None).replace("ET", None).replace("EP", None).replace("", None)

            columnNames = ['BIM_1','BIM_2']

            df1.columns = columnNames

            #EXCLUIR MATÉRIAS - NOTAS DO ALUNO
            df2 = df1.drop('TOTAIS').drop('ENSINO MEDIO').drop('ED. FISICA')

            #print(df2)
            #if page == pdf.pages[0]:
            #    print(1)

            #REMOÇÃO DE STRINGS E NULOS

            df3 = df2.apply(pd.to_numeric, errors='coerce')

            mean_by_bim = df3
            mean_stu = df3.stack(level=-1, dropna=True).mean()
            
            

            temp_df = pd.DataFrame({'TURMA': [stu_class], 'NOME': [name_A], 'RA':[ra_A], 'MEDIA':[mean_stu]}) 

            base_df = pd.concat((base_df, temp_df) , axis = 0)
        
        


A primeira etapa consolida as notas de todos os alunos. BASE_DF já considera a média de todos eles

Na segunda etapa é possível que alguns alunos tenham mudado de turma (mudança de horário). O código abaixo remove o alunos que apresentam notas em mais de uma turma.

In [6]:
#CONTANDO QUANTAS VEZES O ALUNO APARECE NO TOTAL
astu = base_df.groupby("NOME").count().rename({'RA': 'VEZES'}, axis=1)

#JUNTANDO COM A BASE FINAL
dfNew = base_df.merge(astu, on = ['NOME'],
             how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#SELECIONANDO APENAS ALUNOS COM NOME DUPLICADO
dss = dfNew.where(dfNew['VEZES']>1).dropna()
#double_stu = temp_df[temp_df['NOME'].isin(dss.index)]

#display(dss.sort_values('NOME'))

O código a seguir remove esses alunos com turma duplicada

In [7]:
base_df_removed_stu = base_df[~base_df['NOME'].isin((dss['NOME']))]


Essa etapa considera apenas alunos com turmas duplicadas. A ideia é listar todos os alunos e as matérias em sequência, somando as notas de cada matéria e de cada bimestre. 

In [ ]:
temp_double_students = pd.DataFrame(columns=['TURMA','NOME', 'RA', 'BIM_1', 'BIM_2'])
double_students = pd.DataFrame(columns=['NOME', 'RA', 'MÉDIA'])

for file in files:
    stu_class = file[15:17]
    
    with pdfplumber.open(file) as pdf:

        for page in pdf.pages[0:]:
            table = page.extract_table(table_settings={"vertical_strategy": "lines", 
                                                       "horizontal_strategy": "lines", 
                                                       "snap_tolerance": 3})

            ################# EXTRAÇÃO DE NOMES E RA #####################

            text = page.extract_text()

            for line in text.split('\n'):
                name = name_re.search(line)
                ra = ra_re.search(line)

                if name:
                    name_A = name.group(2) 

                if ra:
                    ra_A = str(ra.group(2))
                    
            df = pd.DataFrame(table, columns=table[0]).T
            
            if name_A  in list(dss['NOME']):
                df = pd.DataFrame(table, columns=table[0]).T
            
                #### OS BIMESTRES SÃO SELECIONADOS AQUI ###
                df1 = df.iloc[3:, 10:12].replace("ES", None).replace("ET", None).replace("EP", None).replace("", None)

                ['BIM_1','BIM_2']

                df1.columns = columnNames

                #EXCLUIR MATÉRIAS - NOTAS DO ALUNO
                df2 = df1.drop('TOTAIS').drop('ENSINO MEDIO').drop('ED. FISICA')

                #REMOÇÃO DE STRINGS E NULOS

                df3 = df2.apply(pd.to_numeric, errors='coerce')

                mean_by_bim = df3
                
                mean_by_bim['TURMA'] = stu_class
                
                mean_by_bim['NOME'] = name_A
                
                mean_by_bim['RA'] = ra_A
                                
                temp_double_students = pd.concat((temp_double_students, mean_by_bim) , axis = 0)
                

O que foi encontrado é que muitas vezes, após a mudança de turma, um aluno fica com nota zero na turma inicial, e na turma final fica com a nota correta. Por isso, as notas são agrupadas por matéria e somadas. Resultando em uma máteria única para cada aluno. Por fim, é realizada a média de todas as notas.

O sum(min_count=1) possibilita a soma de N/As resultar em N/As, isso é muito importante, porque sem isso a soma de N/A resulta em 0, e na média 0 é considerado, enquanto N/A não. Apenas matérias básicas fazem parte do cálculo final, i.e., Matémática, Filosofia, português, Química, física e etc.

In [ ]:
#display(temp_double_students)
ds = temp_double_students
ds_ri = ds.reset_index()
ds_grouped_class = ds_ri.groupby(['NOME', 'RA', 'index'], as_index = False).agg({'TURMA': ', '.join})

ds_grouped_subjects = ds_ri.groupby(['NOME','index', 'RA']).sum(min_count=1)
ds_grouped_subjects_ri = ds_grouped_subjects.reset_index(drop=False)
ds_grouped_subject_final = ds_grouped_subjects_ri.drop('TURMA', axis = 1)

ds_subject_class = ds_grouped_subject_final.merge(ds_grouped_class, on=['NOME', 'RA', 'index'],
             how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

ds_subject_class_sum = ds_subject_class.groupby(['TURMA', 'NOME','index', 'RA']).sum(min_count=1)

ds_subject_class_mean = ds_subject_class_sum.mean(axis = 1)

ds_final = ds_subject_class_mean.groupby(['TURMA','NOME', 'RA']).mean().reset_index(name ='MEDIA')

ds_final_notna = ds_final[ds_final['MEDIA'].notna()]

#display(ds_final_notna)

In [ ]:
#r = temp_double_students
#r2 = r.reset_index()
#r3 = r2.groupby(['TURMA', 'NOME','index', 'RA']).sum(min_count=1)


#r4 = r3.mean(axis = 1)

#r5 = r4.groupby(['TURMA','NOME', 'RA']).mean().reset_index(name ='MEDIA')


In [ ]:
base_final = pd.concat((base_df_removed_stu, ds_final_notna) , axis = 0)

In [ ]:
base_final.to_csv('medias_cdt_2022_sem1.csv', sep='\t')

In [ ]:
display(base_df.loc[base_df['NOME'] == 'Nome Aluno'])